In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='data/russian/'

In [3]:
!ls {PATH}

item_categories.csv      sales_train.csv.gz       test.csv
items.csv                sample_submission.csv    test.csv.gz
joined                   sample_submission.csv.gz
sales_train.csv          shops.csv


In [4]:
import os

In [5]:
import pandas as pd

In [7]:
subm = pd.read_csv(f'{PATH}sample_submission.csv')
subm.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [8]:
test = pd.read_csv(f'{PATH}test.csv')
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [44]:
subm.shape,test.shape

((214200, 2), (214200, 3))

In [8]:
tables = [pd.read_csv(f'{PATH}{f}.csv', low_memory=False) for f in ['item_categories', 'items', 'shops', 'sales_train']]

In [63]:
for t in tables: display(t.head())

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [10]:
for t in tables: display(DataFrameSummary(t).summary())

,item_category_name,item_category_id
count,NaN,84
mean,NaN,41.5
std,NaN,24.3926
min,NaN,0
25%,NaN,20.75
50%,NaN,41.5
75%,NaN,62.25
max,NaN,83
counts,84,84
uniques,84,84


,item_name,item_id,item_category_id
count,NaN,22170,22170
mean,NaN,11084.5,46.2908
std,NaN,6400.07,15.9415
min,NaN,0,0
25%,NaN,5542.25,37
50%,NaN,11084.5,40
75%,NaN,16626.8,58
max,NaN,22169,83
counts,22170,22170,22170
uniques,22170,22170,84


,shop_name,shop_id
count,NaN,60
mean,NaN,29.5
std,NaN,17.4642
min,NaN,0
25%,NaN,14.75
50%,NaN,29.5
75%,NaN,44.25
max,NaN,59
counts,60,60
uniques,60,60


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,NaN,2.93585e+06,2.93585e+06,2.93585e+06,2.93585e+06,2.93585e+06
mean,NaN,14.5699,33.0017,10197.2,890.853,1.24264
std,NaN,9.42299,16.227,6324.3,1729.8,2.61883
min,NaN,0,0,0,-1,-22
25%,NaN,7,22,4476,249,1
50%,NaN,14,31,9343,399,1
75%,NaN,23,47,15684,999,1
max,NaN,33,59,22169,307980,2169
counts,2935849,2935849,2935849,2935849,2935849,2935849
uniques,1034,34,60,21807,19993,198


In [64]:
item_cat, items, shops, train = tables

# Join tables
- Drop 'item_name' and we don't need 'item_category_name' since they are represented by 'item_id' and 'item_category_id' respectively
- Dropping date column to start with the simplest data

In [65]:
jointrain = pd.merge(train,items,how='left',on='item_id')

In [67]:
jointrain.drop(['item_name','date'],axis=1,inplace=True)

In [69]:
display(jointrain.head())
display(DataFrameSummary(jointrain).summary())

,date_block_num,shop_id,item_id,item_category_id,item_price,item_cnt_day
0,0,59,22154,37,999.00,1.0
1,0,25,2552,58,899.00,1.0
2,0,25,2552,58,899.00,-1.0
3,0,25,2554,58,1709.05,1.0
4,0,25,2555,56,1099.00,1.0


,date_block_num,shop_id,item_id,item_category_id,item_price,item_cnt_day
count,2.93585e+06,2.93585e+06,2.93585e+06,2.93585e+06,2.93585e+06,2.93585e+06
mean,14.5699,33.0017,10197.2,40.0014,890.853,1.24264
std,9.42299,16.227,6324.3,17.1008,1729.8,2.61883
min,0,0,0,0,-1,-22
25%,7,22,4476,28,249,1
50%,14,31,9343,40,399,1
75%,23,47,15684,55,999,1
max,33,59,22169,83,307980,2169
counts,2935849,2935849,2935849,2935849,2935849,2935849
uniques,34,60,21807,84,19993,198


In [70]:
jointrain.to_feather(f'{PATH}joined')

In [71]:
!ls {PATH}

item_categories.csv      sales_train.csv.gz       test.csv
items.csv                sample_submission.csv    test.csv.gz
joined                   sample_submission.csv.gz
sales_train.csv          shops.csv


In [3]:
train = pd.read_feather(f'{PATH}joined')

In [4]:
train.head()

,date_block_num,shop_id,item_id,item_category_id,item_price,item_cnt_day
0,0,59,22154,37,999.00,1.0
1,0,25,2552,58,899.00,1.0
2,0,25,2552,58,899.00,-1.0
3,0,25,2554,58,1709.05,1.0
4,0,25,2555,56,1099.00,1.0


# Group by month

In [5]:
grouped = train.groupby(by=['date_block_num','shop_id','item_id'], sort=False)

In [6]:
traingroup = grouped.agg({'item_price':np.mean, 'item_cnt_day':np.sum, 'item_category_id':np.mean}).reset_index().rename(columns={'item_cnt_day':'item_cnt_month'})
DataFrameSummary(traingroup).summary()

,date_block_num,shop_id,item_id,item_price,item_cnt_month,item_category_id
count,1.60912e+06,1.60912e+06,1.60912e+06,1.60912e+06,1.60912e+06,1.60912e+06
mean,14.6648,32.8058,10681,790.885,2.2672,41.5419
std,9.54232,16.537,6238.88,1551.61,8.64988,16.3237
min,0,0,0,0.09,-22,0
25%,6,21,5045,199,1,30
50%,14,31,10497,399,1,40
75%,23,47,16060,898.5,2,55
max,33,59,22169,307980,2253,83
counts,1609124,1609124,1609124,1609124,1609124,1609124
uniques,34,60,21807,41464,446,84


In [7]:
cat_vars = ['date_block_num', 'shop_id', 'item_id', 'item_category_id']

In [8]:
for column in cat_vars: traingroup[column] = traingroup[column].astype('category')
#Rearrange columns
traingroup = traingroup[['date_block_num', 'shop_id', 'item_id', 'item_category_id', 'item_price', 'item_cnt_month']]
display(traingroup.head())
display(traingroup.tail())
DataFrameSummary(traingroup).summary()

,date_block_num,shop_id,item_id,item_category_id,item_price,item_cnt_month
0,0,59,22154,37,999.00,1.0
1,0,25,2552,58,899.00,0.0
2,0,25,2554,58,1709.05,1.0
3,0,25,2555,56,1099.00,1.0
4,0,25,2564,59,349.00,1.0


,date_block_num,shop_id,item_id,item_category_id,item_price,item_cnt_month
1609119,33,25,7409,55,299.0,3.0
1609120,33,25,7393,55,349.0,1.0
1609121,33,25,7384,55,749.0,1.0
1609122,33,25,7459,55,349.0,1.0
1609123,33,25,7440,57,299.0,1.0


,date_block_num,shop_id,item_id,item_category_id,item_price,item_cnt_month
count,NaN,NaN,NaN,NaN,1.60912e+06,1.60912e+06
mean,NaN,NaN,NaN,NaN,790.885,2.2672
std,NaN,NaN,NaN,NaN,1551.61,8.64988
min,NaN,NaN,NaN,NaN,0.09,-22
25%,NaN,NaN,NaN,NaN,199,1
50%,NaN,NaN,NaN,NaN,399,1
75%,NaN,NaN,NaN,NaN,898.5,2
max,NaN,NaN,NaN,NaN,307980,2253
counts,1609124,1609124,1609124,1609124,1609124,1609124
uniques,34,60,21807,84,41464,446


# Get validation set
- The last month will be the validation set

In [9]:
val_idx = traingroup[traingroup.date_block_num == 33].index
val_idx

Int64Index([1577593, 1577594, 1577595, 1577596, 1577597, 1577598, 1577599,
            1577600, 1577601, 1577602,
            ...
            1609114, 1609115, 1609116, 1609117, 1609118, 1609119, 1609120,
            1609121, 1609122, 1609123],
           dtype='int64', length=31531)

# Prepare data for DL

In [10]:
df, y, nas, mapper = proc_df(traingroup, 'item_cnt_month', do_scale=True)

In [11]:
yl = np.log(y).astype(np.float32)
y, yl

/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


(array([ 1.,  0.,  1.,  1.,  1.,  1.,  2.,  3., 13.,  1.,  2.,  2.,  1.,  1.,  1.,  1.,  1.,  4.,  2.,  2.,
        ...,  1.,  1.,  1.,  1.,  1.,  3.,  2.,  1.,  1.,  1.,  1.,  2.,  1.,  1.,  1.,  3.,  1.,  1.,  1.,
         1.]),
 array([0.     ,    -inf, 0.     , 0.     , 0.     , 0.     , 0.69315, 1.09861, 2.56495, 0.     , 0.69315,
        0.69315, 0.     , 0.     , 0.     , 0.     , 0.     , 1.38629, 0.69315, 0.69315, ..., 0.     ,
        0.     , 0.     , 0.     , 0.     , 1.09861, 0.69315, 0.     , 0.     , 0.     , 0.     , 0.69315,
        0.     , 0.     , 0.     , 1.09861, 0.     , 0.     , 0.     , 0.     ], dtype=float32))

In [12]:
np.exp(np.log(0))

/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


0.0

In [13]:
mapper

DataFrameMapper(default=False, df_out=False,
        features=[(['item_price'], StandardScaler(copy=True, with_mean=True, with_std=True))],
        input_df=False, sparse=False)

In [14]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = targ - inv_y(y_pred)
    return math.sqrt((pct_var**2).mean())

In [15]:
max_log_y = np.log(np.max(y))
y_range = (0, max_log_y*1.2)
y_range

(0, 9.264021528518693)

# Time to learn

In [16]:
md = ColumnarModelData.from_data_frame(PATH,val_idx, df, yl, cat_vars, bs=128)

In [17]:
cat_sz = [(c, len(traingroup[c].cat.categories)+1) for c in cat_vars]
cat_sz

[('date_block_num', 35),
 ('shop_id', 61),
 ('item_id', 21808),
 ('item_category_id', 85)]

In [18]:
emb_szs = [(c, min(50,(c+1)//2)) for _,c in cat_sz]
emb_szs

[(35, 18), (61, 31), (21808, 50), (85, 43)]

In [19]:
learn = md.get_learner(emb_szs, 1, 0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)

In [20]:
learn.lr_find()

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  0%|          | 1/12325 [00:00<1:31:06,  2.25it/s, loss=inf] 
